# Homework 3: Vaccine Warehouse Binary

###  Bus 36109 "Advanced Decision Modeling with Python", Don Eisenstein
Don Eisenstein &copy; Copyright 2024, University of Chicago

## Instructions

A vaccine is being produced at different manufacturing Plants to be distributed across the country to various Hospitals. Vaccines must travel through Warehouses along the way to a Hospital.

Vaccines are transported in units of cases.  The capacity of a plant is in units of cases per week.  Each Warehouse must process and ship all the cases it receives each week.  Each Warehouse has a capacity of cases it can process each week and a processing cost to process each case.  Each Warehouse also has a fixed weekly operating cost that is incurred if it processes any cases of vaccine.  If a Warehouse processes no vaccine then its fixed cost is not incurred (that is, we can close a warehouse down and not incur its fixed cost).

Each Hospital has a demand for vaccine cases each week.  The supply of vaccine to a hospital must not exceed its demand.  

The transportation cost is \$1 per vaccine case traveling one unit of distance between plants and warehouses and between warehouses and hospitals.

All facilites, distances and costs are described in an Airtable base.

Each manufacturing Plant must ship as many doses as possible in an attempt to meet, but not exceed, Hospital demand.  A manufacturing Plant can ship vaccines to multiple Warehouses, a Warehouse can ship vaccines to multiple Hospitals, and a Hospital can receive vaccines from multiple Warehouses.   

Your code should be robust, that is, it should NOT make any assumptions about the total plant and warehouse capacities, or total hospital demands.

Your model should find the optimal vaccine flows from Plants to Warehouses and Warehouses to Hospitals to minimize the weekly transportation, processing and operating costs, while sending as much vaccine as is feasibly possible.  And in doing so, determining which Warehouses are Open and which Closed.

**IMPORTANT:** Model your flow of vaccine cases as a continuous variable.  That is, you can ship fractions of cases between facilities.

**NOTE** No single variable carrying flow should include the ENTIRE path from a Plant through a Warehouse and into a Hospital.   That is, you must have a set of simple flow variables from Plants to Warehouses, and another set of simple flow variables from Warehouses to Hospitals.

Follow the notebook to walk you through the solution in parts.  Insert your answer to each part into the notebook below the question for each part.   Turn in your completed notebook with all output visible.

# Your Solution

Insert your answer to each part into this notebook

In [1]:
! pip install pyairtable

In [2]:
! pip install pulp

In [3]:
import pulp
from pulp import LpBinary
from pyairtable import Api
from pprint import pprint

In [4]:
AIRTABLE_API_TOKEN= "patvwCWqLLfjG2BBl.4407710f167dc945893b8d08cc262236311b59baf23106c522ad9138a7b4c2f6"

BASE_ID="appWITdHvwKWupWZR" # The ID for the HW3 data table -- don't change this

api = Api(AIRTABLE_API_TOKEN)

PLANT_TABLE_ID = 'plants'
WAREHOUSES_TABLE_ID = 'warehouses'
HOSPITALS_TABLE_ID = 'hospitals'
DISTANCE_TABLE_ID = 'distances'

plants_table = api.table(BASE_ID, PLANT_TABLE_ID)
warehouses_table = api.table(BASE_ID, WAREHOUSES_TABLE_ID)
hospitals_table = api.table(BASE_ID, HOSPITALS_TABLE_ID)
distances_table = api.table(BASE_ID, DISTANCE_TABLE_ID)

<!-- BEGIN QUESTION -->

**1. In broad terms, what are the variables, objective and constraints of this problem? You don't need to list the entire formulation. Just describe the structure/characteristics of your model.**

Variables:
There are several variables including:

*   amount from plant to warehouse
*   amount from warehouse to hospital
*   demand of vaccines from hospital
*   unit / transportation / distance / processing cost



Constraints:


*   Capacity of Each Plant
*   Capacity of Each Warehouse
*  Vaccines must flow from plant -> warehouse -> hospital
* Can turn warehouses on and off



**2. Click on this link to access the data on AirTable: [AirTable Data](https://airtable.com/invite/l?inviteId=invCF2IEwn3KB68tR&inviteToken=8feebbc273e675e663a026e7a321221cedbdd4d893229610e2df40a2488647ef&utm_medium=email&utm_source=product_team&utm_content=transactional-alerts)**

You may have to add access to this airtable base onto your token if you did not add permission to ALL bases to start.

<!-- END QUESTION -->

**3. Read in the `plants` table from AirTable. Store in an appropriate structure in Python.  Print out your Python structure. Verify that the data looks as expected.**

In [5]:
plants = []

for plant in plants_table.all():
    plants.append(plant['fields'])

In [6]:
plants

[{'name': 'plant_6', 'capacity': 100},
 {'name': 'plant_5', 'capacity': 100},
 {'name': 'plant_2', 'capacity': 60},
 {'name': 'plant_3', 'capacity': 150},
 {'name': 'plant_1', 'capacity': 200},
 {'name': 'plant_4', 'capacity': 40}]

**4.  Now read in, print, and verify the hospitals table.**

In [7]:
hospitals = []

for hospital in hospitals_table.all():
    hospitals.append(hospital['fields'])

In [8]:
hospitals

[{'name': 'hospital_2', 'demand': 40},
 {'name': 'hospital_10', 'demand': 70},
 {'name': 'hospital_6', 'demand': 90},
 {'name': 'hospital_1', 'demand': 50},
 {'name': 'hospital_4', 'demand': 120},
 {'name': 'hospital_7', 'demand': 100},
 {'name': 'hospital_5', 'demand': 50},
 {'name': 'hospital_8', 'demand': 90},
 {'name': 'hospital_9', 'demand': 50},
 {'name': 'hospital_3', 'demand': 90}]

**5.  Now read in, print, and verify the warehouse table.**

In [9]:
warehouses = []

for warehouse in warehouses_table.all():
    warehouses.append(warehouse['fields'])

In [10]:
warehouses

[{'name': 'warehouse_8',
  'capacity': 100,
  'processing_cost': 10,
  'fixed_cost': 550},
 {'name': 'warehouse_5',
  'capacity': 80,
  'processing_cost': 25,
  'fixed_cost': 1200},
 {'name': 'warehouse_4',
  'capacity': 100,
  'processing_cost': 30,
  'fixed_cost': 310},
 {'name': 'warehouse_6',
  'capacity': 290,
  'processing_cost': 10,
  'fixed_cost': 1590},
 {'name': 'warehouse_3',
  'capacity': 80,
  'processing_cost': 30,
  'fixed_cost': 290},
 {'name': 'warehouse_7',
  'capacity': 50,
  'processing_cost': 25,
  'fixed_cost': 410},
 {'name': 'warehouse_2',
  'capacity': 200,
  'processing_cost': 25,
  'fixed_cost': 750},
 {'name': 'warehouse_1',
  'capacity': 200,
  'processing_cost': 20,
  'fixed_cost': 1000}]

**6.  Now read in, print, and verify the distances table.**

In [11]:
distances = []

for distance in distances_table.all():
    distances.append(distance['fields'])

In [12]:
distances

[{'start': 'warehouse_5', 'end': 'hospital_3', 'distance': 284},
 {'start': 'warehouse_3', 'end': 'hospital_9', 'distance': 269},
 {'start': 'plant_2', 'end': 'warehouse_3', 'distance': 106},
 {'start': 'plant_3', 'end': 'warehouse_6', 'distance': 70},
 {'start': 'warehouse_5', 'end': 'hospital_5', 'distance': 213},
 {'start': 'warehouse_7', 'end': 'hospital_5', 'distance': 228},
 {'start': 'warehouse_4', 'end': 'hospital_1', 'distance': 364},
 {'start': 'warehouse_5', 'end': 'hospital_9', 'distance': 209},
 {'start': 'warehouse_6', 'end': 'hospital_4', 'distance': 125},
 {'start': 'plant_6', 'end': 'warehouse_6', 'distance': 315},
 {'start': 'plant_1', 'end': 'warehouse_6', 'distance': 177},
 {'start': 'warehouse_4', 'end': 'hospital_3', 'distance': 191},
 {'start': 'warehouse_7', 'end': 'hospital_6', 'distance': 110},
 {'start': 'warehouse_8', 'end': 'hospital_4', 'distance': 121},
 {'start': 'warehouse_3', 'end': 'hospital_2', 'distance': 10},
 {'start': 'warehouse_6', 'end': 'hospi

**7. Create a PuLP LpProblem object and store it in the variable `model`.**

In [13]:
model = pulp.LpProblem("Vax_Watehouse_HW3",pulp.LpMinimize)

**8. Create and store your PuLP decision variables. Print out your Python structures that hold them.**


In [14]:
#names -------------
lp_variables_p_w = {}

for plant in plants:
    for warehouse in warehouses:
        # Create a dictionary entry referencing the decision variable for this plant-hospital pair
        lp_variables_p_w[(plant['name'], warehouse['name'])] = pulp.LpVariable(f"{plant['name']}_{warehouse['name']}", cat='Continuous', lowBound=0)

lp_variables_w_h = {}

for warehouse in warehouses:
    for hospital in hospitals:
        # Create a dictionary entry referencing the decision variable for this plant-hospital pair
        lp_variables_w_h[(warehouse['name'], hospital['name'])] = pulp.LpVariable(f"{warehouse['name']}_{hospital['name']}", cat='Continuous', lowBound=0)

#distances -------------
plant_to_wh_dist = {}

wh_to_hosp_dist = {}

for distance in distances:
    start_location = distance['start']
    end_location = distance['end']
    distance_v = distance['distance']

    # Check against actual plant and warehouse names via looping
    for plant in plants:
        for warehouse in warehouses:
            if start_location == plant['name'] and end_location == warehouse['name']:
                plant_to_wh_dist[(plant['name'], warehouse['name'])] = distance_v

    for warehouse in warehouses:
        for hospital in hospitals:
            if start_location == warehouse['name'] and end_location == hospital['name']:
                wh_to_hosp_dist[(warehouse['name'], hospital['name'])] = distance_v

open_warehouse = {}
for warehouse in warehouses:
    open_warehouse[warehouse['name']] = pulp.LpVariable(f"open_{warehouse['name']}", cat='Binary')

obj_plant_to_wh = 0

for plant, warehouse in lp_variables_p_w:
  obj_plant_to_wh += lp_variables_p_w[(plant, warehouse)] * plant_to_wh_dist[(plant, warehouse)]

obj_wh_to_hosp = 0

for warehouse, hospital in lp_variables_w_h:
    obj_wh_to_hosp += lp_variables_w_h[(warehouse, hospital)] * wh_to_hosp_dist[(warehouse, hospital)]

obj_proc_cost = 0

for warehouse in warehouses:
    for hospital in hospitals:
        dict_key = (warehouse['name'], hospital['name'])
        if dict_key in lp_variables_w_h:
            obj_proc_cost += lp_variables_w_h[dict_key] * warehouse['processing_cost']

obj_fix_cost = 0

for warehouse in warehouses:
    obj_fix_cost += open_warehouse[warehouse['name']] * warehouse['fixed_cost']

objective = obj_plant_to_wh + obj_wh_to_hosp + obj_proc_cost + obj_fix_cost

In [15]:
objective

1000*open_warehouse_1 + 750*open_warehouse_2 + 290*open_warehouse_3 + 310*open_warehouse_4 + 1200*open_warehouse_5 + 1590*open_warehouse_6 + 410*open_warehouse_7 + 550*open_warehouse_8 + 203*plant_1_warehouse_1 + 270*plant_1_warehouse_2 + 162*plant_1_warehouse_3 + 129*plant_1_warehouse_4 + 222*plant_1_warehouse_5 + 177*plant_1_warehouse_6 + 17*plant_1_warehouse_7 + 213*plant_1_warehouse_8 + 19*plant_2_warehouse_1 + 214*plant_2_warehouse_2 + 106*plant_2_warehouse_3 + 315*plant_2_warehouse_4 + 166*plant_2_warehouse_5 + 63*plant_2_warehouse_6 + 219*plant_2_warehouse_7 + 157*plant_2_warehouse_8 + 138*plant_3_warehouse_1 + 347*plant_3_warehouse_2 + 239*plant_3_warehouse_3 + 220*plant_3_warehouse_4 + 299*plant_3_warehouse_5 + 70*plant_3_warehouse_6 + 124*plant_3_warehouse_7 + 290*plant_3_warehouse_8 + 98*plant_4_warehouse_1 + 111*plant_4_warehouse_2 + 13*plant_4_warehouse_3 + 240*plant_4_warehouse_4 + 63*plant_4_warehouse_5 + 166*plant_4_warehouse_6 + 144*plant_4_warehouse_7 + 60*plant_4_war

**9. Add your objective function to your `model`.  Print your model to verify**

In [16]:
model += objective, 'Minimize transportation cost'

print(model)

Vax_Watehouse_HW3:
MINIMIZE
1000*open_warehouse_1 + 750*open_warehouse_2 + 290*open_warehouse_3 + 310*open_warehouse_4 + 1200*open_warehouse_5 + 1590*open_warehouse_6 + 410*open_warehouse_7 + 550*open_warehouse_8 + 203*plant_1_warehouse_1 + 270*plant_1_warehouse_2 + 162*plant_1_warehouse_3 + 129*plant_1_warehouse_4 + 222*plant_1_warehouse_5 + 177*plant_1_warehouse_6 + 17*plant_1_warehouse_7 + 213*plant_1_warehouse_8 + 19*plant_2_warehouse_1 + 214*plant_2_warehouse_2 + 106*plant_2_warehouse_3 + 315*plant_2_warehouse_4 + 166*plant_2_warehouse_5 + 63*plant_2_warehouse_6 + 219*plant_2_warehouse_7 + 157*plant_2_warehouse_8 + 138*plant_3_warehouse_1 + 347*plant_3_warehouse_2 + 239*plant_3_warehouse_3 + 220*plant_3_warehouse_4 + 299*plant_3_warehouse_5 + 70*plant_3_warehouse_6 + 124*plant_3_warehouse_7 + 290*plant_3_warehouse_8 + 98*plant_4_warehouse_1 + 111*plant_4_warehouse_2 + 13*plant_4_warehouse_3 + 240*plant_4_warehouse_4 + 63*plant_4_warehouse_5 + 166*plant_4_warehouse_6 + 144*plant_4_

**10. Add all of your constraints to your `model`.   Use python comments to document each type of constraint before you add them.**

In [17]:
# Plant capacity
for plant in plants:
    for warehouse in warehouses:
        if (plant['name'], warehouse['name']) in lp_variables_p_w:
            model += (
                lp_variables_p_w[(plant['name'], warehouse['name'])] <= plant['capacity'],
                f"plant_capacity_{plant['name']}_to_{warehouse['name']}"
            )

# Warehouse capacity
for warehouse in warehouses:
    for hospital in hospitals:
        if (warehouse['name'], hospital['name']) in lp_variables_w_h:
            model += (
                lp_variables_w_h[(warehouse['name'], hospital['name'])] <= warehouse['capacity'] * open_warehouse[warehouse['name']],
                f"warehouse_capacity_{warehouse['name']}_to_{hospital['name']}"
            )

# Warehouse vaccine in and out


for warehouse in warehouses:
    vac_in = 0
    vac_out = 0
    for plant in plants:
        if (plant['name'], warehouse['name']) in lp_variables_p_w:
            vac_in += lp_variables_p_w[(plant['name'], warehouse['name'])]
    for hospital in hospitals:
        if (warehouse['name'], hospital['name']) in lp_variables_w_h:
            vac_out += lp_variables_w_h[(warehouse['name'], hospital['name'])]
    model += (vac_in == vac_out, f"balance_{warehouse['name']}")

# Total flow of vaccines should be equal to the total available vaccines
total_vaccines = 0

for plant in plants:
    total_vaccines += plant['capacity']

total_distributed_vaccines = 0
for warehouse in warehouses:
    for hospital in hospitals:
        if (warehouse['name'], hospital['name']) in lp_variables_w_h:
            total_distributed_vaccines += lp_variables_w_h[(warehouse['name'], hospital['name'])]

model += (total_distributed_vaccines == total_vaccines, "total_vaccines")

# Ensure that no hospital receives more vaccines than its demand
for hospital in hospitals:
    total_flow_to_hospital = 0
    for warehouse in warehouses:
        if (warehouse['name'], hospital['name']) in lp_variables_w_h:
            total_flow_to_hospital += lp_variables_w_h[(warehouse['name'], hospital['name'])]
    model += (
        total_flow_to_hospital <= hospital['demand'],
        f"hospital_demand_{hospital['name']}"
    )

# Only allow flow through warehouse if it's open


for warehouse in warehouses:
    for hospital in hospitals:
        if (warehouse['name'], hospital['name']) in lp_variables_w_h:
            model += (
                lp_variables_w_h[(warehouse['name'], hospital['name'])] <= warehouse['capacity'] * open_warehouse[warehouse['name']],
                f"warehouse_open_{warehouse['name']}_to_{hospital['name']}"
            )

<!-- BEGIN QUESTION -->

**11. Display your model with `print(model)`, check that all is OK**

In [18]:
print(model)

Vax_Watehouse_HW3:
MINIMIZE
1000*open_warehouse_1 + 750*open_warehouse_2 + 290*open_warehouse_3 + 310*open_warehouse_4 + 1200*open_warehouse_5 + 1590*open_warehouse_6 + 410*open_warehouse_7 + 550*open_warehouse_8 + 203*plant_1_warehouse_1 + 270*plant_1_warehouse_2 + 162*plant_1_warehouse_3 + 129*plant_1_warehouse_4 + 222*plant_1_warehouse_5 + 177*plant_1_warehouse_6 + 17*plant_1_warehouse_7 + 213*plant_1_warehouse_8 + 19*plant_2_warehouse_1 + 214*plant_2_warehouse_2 + 106*plant_2_warehouse_3 + 315*plant_2_warehouse_4 + 166*plant_2_warehouse_5 + 63*plant_2_warehouse_6 + 219*plant_2_warehouse_7 + 157*plant_2_warehouse_8 + 138*plant_3_warehouse_1 + 347*plant_3_warehouse_2 + 239*plant_3_warehouse_3 + 220*plant_3_warehouse_4 + 299*plant_3_warehouse_5 + 70*plant_3_warehouse_6 + 124*plant_3_warehouse_7 + 290*plant_3_warehouse_8 + 98*plant_4_warehouse_1 + 111*plant_4_warehouse_2 + 13*plant_4_warehouse_3 + 240*plant_4_warehouse_4 + 63*plant_4_warehouse_5 + 166*plant_4_warehouse_6 + 144*plant_4_

<!-- END QUESTION -->

**12. Solve your optimization model and print its status and the optimal objective function value.  The optimal objective function value is 117910.0**

In [19]:
model.solve()
print("Status:", pulp.LpStatus[model.status])
if (pulp.LpStatus[model.status] == 'Optimal'):
    print("Obj function value:", model.objective.value(), "\n")

Status: Optimal
Obj function value: 93840.0 



**13. Output the value of each of your variables at optimality.  Only print variables with non-zero value.**

In [20]:
for v in model.variables():
    if v.varValue > 0:
        print(v, v.varValue)

open_warehouse_1 1.0
open_warehouse_2 1.0
open_warehouse_3 1.0
open_warehouse_4 1.0
open_warehouse_6 1.0
open_warehouse_7 1.0
open_warehouse_8 1.0
plant_1_warehouse_7 150.0
plant_2_warehouse_1 60.0
plant_2_warehouse_3 20.0
plant_2_warehouse_6 60.0
plant_3_warehouse_6 120.0
plant_4_warehouse_1 40.0
plant_4_warehouse_3 40.0
plant_4_warehouse_8 40.0
plant_5_warehouse_2 50.0
plant_6_warehouse_4 70.0
warehouse_1_hospital_4 100.0
warehouse_2_hospital_8 50.0
warehouse_3_hospital_2 40.0
warehouse_3_hospital_4 20.0
warehouse_4_hospital_10 70.0
warehouse_6_hospital_1 50.0
warehouse_6_hospital_3 30.0
warehouse_6_hospital_5 50.0
warehouse_6_hospital_7 50.0
warehouse_7_hospital_3 50.0
warehouse_7_hospital_6 50.0
warehouse_7_hospital_7 50.0
warehouse_8_hospital_8 40.0


**14. Use Python to loop through each Hospital, display the total vaccine supplied to each hospital, its demand, shorfall, and percent of demand met.**

In [30]:
for hospital in hospitals:
    name = hospital['name']
    demand = hospital['demand']
    constraint_name = f"hospital_demand_{name}"
    slack = model.constraints[constraint_name].slack
    supply = demand - slack
    percent_met = (supply / demand) * 100 if demand > 0 else 0
    print(f"{name:<20} {supply:10.2f} {demand:10.2f} {slack:12.2f} {percent_met:10.2f}%")

hospital_2                40.00      40.00        -0.00     100.00%
hospital_10               70.00      70.00        -0.00     100.00%
hospital_6                50.00      90.00        40.00      55.56%
hospital_1                50.00      50.00        -0.00     100.00%
hospital_4               120.00     120.00        -0.00     100.00%
hospital_7               100.00     100.00        -0.00     100.00%
hospital_5                50.00      50.00        -0.00     100.00%
hospital_8                90.00      90.00        -0.00     100.00%
hospital_9                 0.00      50.00        50.00       0.00%
hospital_3                80.00      90.00        10.00      88.89%
